<a href="https://colab.research.google.com/github/RushikeshBhabad/GenAi/blob/main/LLAMA_70B_Rag_ChromaDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install chromadb==0.5.5 langchain-chroma==0.1.2 langchain==0.2.11 langchain-community==0.2.10 langchain-text-splitters==0.2.2 langchain-groq==0.1.6 transformers==4.43.2 sentence-transformers==3.0.1 unstructured==0.15.0 unstructured[pdf]==0.15.0

In [ ]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 36.0 MB/s eta 0:00:00


In [ ]:
import os

from langchain.document_loaders import UnstructuredFileLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

In [ ]:
os.environ["GROQ_API_KEY"] = "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"

In [ ]:
# Loading the document
from langchain.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("/Data-Science.pdf")
documents = loader.load()

In [ ]:
print(documents)

[Document(metadata={'source': '/Data-Science.pdf', 'file_path': '/Data-Science.pdf', 'page': 0, 'total_pages': 14, 'format': 'PDF 1.5', 'title': '', 'author': 'sheetal', 'subject': '', 'keywords': '', 'creator': 'Microsoft® Office Word 2007', 'producer': 'Microsoft® Office Word 2007', 'creationDate': 'D:20210405094746Z', 'modDate': "D:20210405153426+05'30'", 'trapped': ''}, page_content=' \n \nFaculty of Science and Technology \nSavitribai Phule Pune University \nMaharashtra, India \n \nhttp://unipune.ac.in \n \nHonours* in Data Science \nBoard of Studies \n (Computer Engineering) \n(With effect from A.Y. 2020-21) \n \n'), Document(metadata={'source': '/Data-Science.pdf', 'file_path': '/Data-Science.pdf', 'page': 1, 'total_pages': 14, 'format': 'PDF 1.5', 'title': '', 'author': 'sheetal', 'subject': '', 'keywords': '', 'creator': 'Microsoft® Office Word 2007', 'producer': 'Microsoft® Office Word 2007', 'creationDate': 'D:20210405094746Z', 'modDate': "D:20210405153426+05'30'", 'trapped'

In [ ]:
# splitting into text chunks
text_splitter = CharacterTextSplitter(
    chunk_size = 2000,
    chunk_overlap = 400 # One chunk is kind of connected to another
)

In [ ]:
texts = text_splitter.split_documents(documents)

In [ ]:
type(texts)

list

In [ ]:
len(texts)

14

In [ ]:
embeddings = HuggingFaceEmbeddings()

In [ ]:
persist_directory = "doc_db"

In [ ]:
vectordb = Chroma.from_documents(
    documents = texts,
    embedding = embeddings,
    persist_directory = persist_directory
)

In [ ]:
# retriever
retriever = vectordb.as_retriever()

In [ ]:
# LLM from groq
llm = ChatGroq(
    model = "llama3-70b-8192",
    temperature = 0
)

In [ ]:
# Create a qa chain
qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = retriever,
    return_source_documents = True
)

In [ ]:
# Invoke QA chain and get a response for a user query
query = "What is the data science subjects ?"
response = qa_chain.invoke({"query": query})

In [ ]:
print(response["result"])

Based on the provided context, the data science subjects in the Savitribai Phule Pune University's Honours in Data Science program are:

1. Data Science and Visualization (310501)
2. Data Science and Visualization Laboratory (310502)
3. Statistics and Machine Learning (310503)
4. Machine Learning and Data Science (410501)
5. Machine Learning and Data Science Laboratory (410502)
6. Artificial Intelligence for Big Data Analytics (410503)
7. Seminar (410504)

These subjects are part of the curriculum for the Honours in Data Science program, spanning across Semesters V to VIII.


In [ ]:
print(response["source_documents"][0].metadata["source"])

/Data-Science.pdf
